In [2]:
#from mlxtend.frequent_patterns import apriori
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [5]:
from scipy.io import arff
data, meta = arff.loadarff('./data/supermarket.arff')

data

array([(b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b't', b't', b'?', b't', b'?', b't', b'?', b'?', b't', b'?', b'?', b'?', b't', b't', b't', b't', b'?', b't', b'?', b't', b't', b'?', b'?', b'?', b'?', b'?', b'?', b't', b't', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b'?', b't', b'?', b'?', b't', b'?', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b'?', b'?', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'